<a href="https://colab.research.google.com/github/manishbayesian/bayesianbookpub/blob/main/Chapter_2_Distributions_with_PyMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2 - Distributions with PyMC

## Imports

In [ ]:
from IPython.display import HTML, display
 
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from scipy import special
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
import numpy as np
import sys

sns.set_theme(style="darkgrid")
pd.set_option('display.precision', 3)
pd.set_option('display.float_format',  '{:,.3f}'.format)

## Install and import PyMC packages

In [ ]:
! pip install pymc

In [ ]:
%env MKL_THREADING_LAYER=GNU
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
import arviz as az
import pymc as pm

In [ ]:
## what version are we using?
print('PyMC Version', pm.__version__)

## What all Distributions are available in PyMC?

### List all distributions in PyMC

In [ ]:
display(pm.distributions.__all__)

#### Code to display all distributions 'nicely'

In [ ]:
distrs = {}
for d in pm.distributions.__all__:
  cls = getattr(sys.modules[pm.distributions.__name__],d)
  cname = str(cls)
  if cname.startswith('<class'):
    dtyp = cname.split("'")[1].split('.')[-2]
    desc = ''
    if cls.__doc__:
      desc = list(filter(lambda x: len(x) > 0, cls.__doc__.split('\n')))[0].strip()
    distrs[d] = {'type':dtyp,'description':desc}

In [ ]:
pd.DataFrame.from_dict(distrs,orient='index')

## Functions for Plotting Distributions & Likelihoods

In [ ]:
def plot_disc(rv, dname = '', ax=None):
    """
    Plot discrete distribution
    """
    if ax is None:
        _, ax = plt.subplots(figsize=(3,3))
    samples = pm.draw(rv,draws=1000)
    x = np.unique(samples)
    y=np.exp(pm.logp(rv, x)).eval()
    ax.stem(x,y,markerfmt='C0o',use_line_collection=True,linefmt='C0-',basefmt=" ") 
    ax.set_xlim([min(x)-0.5,max(x)+0.5])
    #ax.set_ylim([0,1])
    ax.set_xticks(x)
   # ax.set_yticklabels(ax.get_yticklabels(),fontsize='small')
    ax.tick_params(axis='y', labelsize=11)
    ax.tick_params(axis='x', labelsize=11)
   # sns.barplot(x=x, , ax=ax)
    ax.set_xlabel('x',fontsize='medium')
    ax.set_ylabel(r'$f_X(x)$',fontsize='medium')
    ax.set_title(r'PMF for $X\sim ' + '{}'.format(dname) + r'$',fontsize='medium')
    return ax


def plot_cont(rv, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    samples = pm.draw(rv,draws=1000)
    x = np.linspace(np.min(samples), np.max(samples), 1000)
    ax.plot(x, np.exp(pm.logp(rv, x)).eval())
    return ax

In [ ]:
def plot_lik(rvs, x, prange, pname = r'\theta', dname='', ax=None, log=False):
  xx = x
  names = r'$X={}$'.format(x)
  if hasattr(x, "__len__"):
    xx = np.broadcast_to(x, (len(prange),len(x))).T
    names = map(lambda z: r'$X={}$'.format(z),x)
  loglik = pm.logp(rvs, xx).eval()
  lik = np.exp(loglik)
  title = r'Likelihoods for {}'.format(dname)
  dat = lik.T
  if log:
    title = 'Log '+ title
    dat = loglik.T
  if ax is None:
        _, ax = plt.subplots()
  sns.lineplot(data=pd.DataFrame(dat, index=prange, columns = names), ax=ax)
  ax.set(title=title,
         xlabel='$'+pname+'$', ylabel='$P(X|'+pname+')$')

## Discrete Distributions

### Bernoulli Distribution

In [ ]:

fig, ax = plt.subplots(1,3,figsize=(10,3),sharey=True)
plot_disc(pm.Bernoulli.dist(p=0.1),dname='Ber(p=0.1)',ax=ax[0])
plot_disc(pm.Bernoulli.dist(p=0.5),dname='Ber(p=0.5)',ax=ax[1])
plot_disc(pm.Bernoulli.dist(p=0.7),dname='Ber(p=0.7)',ax=ax[2])
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3),sharey=False)
pi = np.linspace(0.001,0.999,100)
logiti = np.linspace(-4,4,100)
plot_lik(pm.Bernoulli.dist(p=pi), [0,1], pi, dname='Bernoulli', pname='p',ax=ax[0])
plot_lik(pm.Bernoulli.dist(logit_p=logiti), [0,1], logiti, dname='Bernoulli', pname='logit(p)',ax=ax[1])




### Binomial Distribution

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),sharey=True)
plot_disc(pm.Binomial.dist(n=6,p=0.5),dname='Bin(n=6,p=0.5)',ax=ax[0])
plot_disc(pm.Binomial.dist(n=6,p=0.7),dname='Bin(n=6,p=0.7)',ax=ax[1])
plot_disc(pm.Binomial.dist(n=12,p=0.7),dname='Bin(n=12,p=0.7)',ax=ax[2])
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3),sharey=False)
pi = np.linspace(0.001,0.999,100)
logiti = np.linspace(-8,8,100)
N = 12
X = [1,4,6,10]
plot_lik(pm.Binomial.dist(n=N,p=pi), X, pi, dname='Binomial', pname='p',ax=ax[0])
plot_lik(pm.Binomial.dist(n=N,logit_p=logiti), X, logiti, dname='Binomial', pname='logit(p)',ax=ax[1])


#### Application of Binomial Distribution: 1-D Random Walk

In [ ]:
### TBD: not sure how to do simple transformations in PyMC

In [ ]:
### switching distributions

In [ ]:
eturn switch(alpha > 1e10,
                      pois,
                      negbinom)

In [ ]:
def get_switched_dist(tau):
  d1 = pm.Uniform("d1", lower = 0, upper = 1.)
  d2 = pm.Uniform("d2", lower = 10., upper = 11.)
  return pm.math.switch(tau > 1.0, d1, d2)

In [ ]:
with pm.Model() as model:
  

In [ ]:

alpha = 1.0/count_data.mean()  # Recall count_data is the
                               # variable that holds our txt counts
lambda_1 = pm.Exponential("lambda_1", alpha)
lambda_2 = pm.Exponential("lambda_2", alpha)

tau = pm.DiscreteUniform("tau", lower=0, upper=n_count_data - 1)

idx = np.arange(n_count_data) # Index
lambda_ = pm.math.switch(tau >= idx, lambda_1,lambda_2)

observation = pm.Poisson("obs", lambda_, observed=count_data)

step = [pm.Metropolis(), pm.NUTS()]
trace = pm.sample(10000, tune=5000,step=step)
pm.traceplot(trace, ['lambda_1', 'lambda_2', 'tau'])
plt.show()

### Categorical Distributions

#### Categorical Distributions as a univariate distribution

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),sharey=True)
plot_disc(pm.Categorical.dist(p=[0.25,0.25,0.25,0.25]),dname='Ber(p=0.1)',ax=ax[0])
plot_disc(pm.Categorical.dist(p=[0.1,0.2,0.3,0.4]),dname='Ber(p=0.5)',ax=ax[1])
plot_disc(pm.Categorical.dist(p=[1/3,1/3,1/3]),dname='Ber(p=0.7)',ax=ax[2])
plt.show()

### Multinomial Distribution - as a Multivariate distribution
For K=3 we will use barycentric coordinates (plot using triangular coordinates). In general, the support of a multinomial distributions with $K$ categories and $N$ trials is on the edges of a $K$-simplex. The vertices represent the points $X=(0,0,...,N), (0,...,N,0),...,(N,0,...,0)$. 

In [ ]:
def plot_3d_bar(X,p):
  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')
  zpos = 0
  dx = dy = 0.5 * np.ones_like(zpos)
  ax.bar3d(X[:,0],X[:,1],zpos,dx,dy,p,zsort='average')

In [ ]:
import plotly.express as px
def plot_scatter_ternary(X,p):
  df = pd.DataFrame(X,columns=['x','y','z'])
  df['P']=p
  fig = px.scatter_ternary(df, a="x", b="y", c="z", color='P', hover_data=['x','y','z','P'],size='P',size_max=12)
  fig.update_layout({
      'ternary': {
          'sum': np.max(X),
          'aaxis': {'title':'$X_0$','tickmode':'linear','tick0':0,'dtick':1},
          'baxis': {'title':'$X_1$','tickmode':'linear','tick0':0,'dtick':1},
          'caxis': {'title':'$X_2$','tickmode':'linear','tick0':0,'dtick':1}
      },
      'height': 400})
  fig.show()

In [ ]:
rv = pm.Multinomial.dist(n=10,p=[4/10,1/10,5/10])
samples = pm.draw(rv,draws=10000)
X = np.unique(samples, axis=0)
y=np.exp(pm.logp(rv, X)).eval()

In [ ]:
plot_scatter_ternary(X,y)

In [ ]:
### We can also plot this multivariate distribution using rectangular coordinates.
plot_3d_bar(X,y)

#### Likelihoods for Multinomial Distrbution

In [ ]:
import plotly.figure_factory as ff
a, b = np.mgrid[0:1:20j, 0:1:20j]
mask = a + b <= 1
a = a[mask].ravel()
b = b[mask].ravel()
c = 1 - a - b
pi = np.stack((a,b,c))


In [ ]:
## get likelihoods
def get_multinomial_lik(n,pi,X):
  rvs = pm.Multinomial.dist(n=10,p=pi.T)
  y=np.exp(pm.logp(rvs, np.asarray(X))).eval()
  return y

In [ ]:
#Xn = X / X.sum(axis=1)[:,np.newaxis]
fig = ff.create_ternary_contour(pi, get_multinomial_lik(10,pi,[2,5,3]))
fig.show()
fig = ff.create_ternary_contour(pi, get_multinomial_lik(10,pi,[1,4,5]))
fig.show()
fig = ff.create_ternary_contour(pi, get_multinomial_lik(10,pi,[0,0,10]))
fig.show()

### Poisson Distribution

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,3),sharey=True,sharex=True)
plot_disc(pm.Poisson.dist(mu=0.1),dname='Pois(\mu=0.1)',ax=ax[0])
plot_disc(pm.Poisson.dist(mu=1),dname='Pois(\mu=1)',ax=ax[1])
plot_disc(pm.Poisson.dist(mu=5),dname='Pois(\mu=5)',ax=ax[2])
ax[0].set(xlim=[-1,11.5])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
mi = np.linspace(1,15,100)
X = [1,4,6,10,12]
plot_lik(pm.Poisson.dist(mu=mi), X, mi, dname='Poisson', pname='\mu',ax=ax)

### Geometric Distribution

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,3),sharey=True,sharex=True)
plot_disc(pm.Geometric.dist(p=0.1),dname='Geom(p=0.1)',ax=ax[0])
plot_disc(pm.Geometric.dist(p=0.5),dname='Geom(p=0.5)',ax=ax[1])
plot_disc(pm.Geometric.dist(p=0.7),dname='Geom(p=0.7)',ax=ax[2])
for a in ax:
  a.set(xlim=[0,10])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3),sharey=False)
pi = np.linspace(0.001,0.999,100)
X = [2,4,8,12]
plot_lik(pm.Geometric.dist(p=pi), X, pi, dname='Geometric', pname='p',ax=ax)

### Negative Binomial Distribution

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,3),sharey=True,sharex=True)
plot_disc(pm.NegativeBinomial.dist(n=4,p=0.2),dname='NegBin(n=4,p=0.2)',ax=ax[0])
plot_disc(pm.NegativeBinomial.dist(n=4,p=0.5),dname='NegBin(n=4,p=0.5)',ax=ax[1])
plot_disc(pm.NegativeBinomial.dist(n=8,p=0.5),dname='NegBin(n=8,p=0.5)',ax=ax[2])
for a in ax:
  a.set(xlim=[-1,30], xticks=range(0,30,4))
  a.tick_params(axis='x', rotation=90)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3),sharey=False)
pi = np.linspace(0.001,0.999,100)
mui = np.linspace(1,20,100)
N = 4
X = [1,2,5,10]
plot_lik(pm.NegativeBinomial.dist(n=N,p=pi), X, pi, dname='Negative Binomial (n=4)', pname='p',ax=ax[0])
plot_lik(pm.NegativeBinomial.dist(n=N,mu=mui), X, mui, dname='Negative Binomial (\alpha=4)', pname='\mu',ax=ax[1])

#### Application: Best of Series. 
We will take best of series of $2n-1$ games between A and B, where A's winning probability for each game is $p$. 
Probability of A winning is $Pr(A) = \sum_{k=0}^{k=n-1}NB(k;n,p)$

We will plot the $Pr(A)$ as a function of $p$ for various values of $n$. Note that as $n$ increases, $Pr(A)$ increases for the same value of $p$ if $p > \frac{1}{2}$ and decreases if $p < \frac{1}{2}$. This means that with a longer series a 'better' team has an even higher chance of winning the series. 

In [ ]:
from scipy.stats import nbinom
def winning_prob_best_of(nseries, p ):
  """ 
  Return probability of winning series, best-of-<nseries>
  where probability of winning each game is p
  """    
  n = int((nseries+1)/2)
  rv = rv = nbinom(n=n, p=p)
  y = 0
  for x in range(0,n):
    y += rv.pmf(x)
  return y


In [ ]:
def winning_prob_best_of(nseries, p ):
  """ 
  Return probability of winning series, best-of-<nseries>
  where probability of winning each game is p
  """    
  n = int((nseries+1)/2)
  rv = pm.NegativeBinomial.dist(n=n,p=p)
  y = 0
  for x in range(0,n):
    y += np.exp(pm.logp(rv, x)).eval()
  return y

In [ ]:
for N in [1,3,7,11,15,19]:
  print('p=0.52, N={}, winning prob={:.03f}'.format(N,winning_prob_best_of(N,0.52)))

In [ ]:
winprob = pd.DataFrame(index=np.arange(0.1,0.9,0.01))
for N in [3,7,15,27,41,77]:
  pA = map(lambda x: winning_prob_best_of(N,x), winprob.index)
  winprob['N={}'.format(N)] = list(pA)

In [ ]:
winprob.plot(figsize=(8,6))




### Hypergeometric Distribution
We notice the PMF is unimodal, and symmetric but as $N$ increases relative to $n$, it starts looking more like a binomial distribution.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,3),sharey=True,sharex=True)
plot_disc(pm.HyperGeometric.dist(N=50,k=10,n=20),dname='HG(N=50,K=10,n=20)',ax=ax[0])
plot_disc(pm.HyperGeometric.dist(N=50,k=25,n=20),dname='HG(N=50,K=25,n=20)',ax=ax[1])
plot_disc(pm.HyperGeometric.dist(N=50,k=25,n=5),dname='HG(N=50,K=25,n=5)',ax=ax[2])
for a in ax:
  a.set(xlim=[-1,16], xticks=range(0,16,2))
  a.tick_params(axis='x', rotation=90)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3),sharey=False)
pi = np.linspace(0.001,0.999,100)
ni = np.linspace(1,50,50)
N = 50
ki = pi*N
X = [1,2,5,10]
plot_lik(pm.HyperGeometric.dist(N=N,k=ki,n=10), X, ki, dname='HypGeom(N=50,n=10)', pname='K',ax=ax[0])
plot_lik(pm.HyperGeometric.dist(N=N,k=25,n=ni), X, ni, dname='HypGeom(N=50,K=25)', pname='n',ax=ax[1])
#plot_lik(pm.NegativeBinomial.dist(n=N,mu=mui), X, mui, dname='Negative Binomial (alpha=4)', pname='\mu',ax=ax[1])

### Extraneous

In [ ]:
plot_cont(n)

In [ ]:
samples = pm.draw(n,draws=10000)
sns.distplot(samples)

In [ ]:
with pm.Model() as nnmodel:
  n1 =  pm.Normal('n1',mu=0, sigma = 1)
  n2 =  pm.Normal('n2',mu=0, sigma = 0.5)
  ss =  pm.Deterministic('quot',n1 / (1+n2))
  ss2 =  pm.Deterministic('sq',n1*n1)
  ssn = n1 + n2
  trace = pm.sample()

In [ ]:
pm.plot_trace(trace)

In [ ]:
samples = pm.draw(nnmodel.sq, draws=1000)
sns.distplot(samples)

In [ ]:
pm.summary(trace)

In [ ]:
with pm.Model():
  nn = pm.Normal('x',mu=0, sigma=1)

In [ ]:
nn.eval()

In [ ]:
import aesara.tensor as at
ss = [at.clip(nn,0,np.inf).eval() for _ in range(1000)]
sns.distplot(np.asarray(ss))

In [ ]:
pm.logp(n,4).eval()

In [ ]:
pm.draw(n, draws=10)

In [ ]:
%env MKL_THREADING_LAYER=GNU
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
import pymc3 as pm3

In [ ]:
n = pm3.Normal.dist(mu=0, sigma=0.5)

In [ ]:
n.random(size=[10,1])

In [ ]:
with pm3.Model() as nnmodel:
  n1 =  pm3.Normal('n1',mu=0, sd = 1)
  n2 =  pm3.Normal('n2',mu=0, sd = 0.5)
  ss =  pm3.Deterministic('sum',n1 + n2)
  ssn = n1 + n2
  trace = pm3.sample(10)

In [ ]:
sn = n + n2